# 1036.逃离大迷宫
## 方法一：有限步数的广度优先搜索
### 思路
判断能否从 $source$ 走到 $target$，最简单且直观的方法就是从 $source$ 开始广度优先搜索，如果搜索过程中经过了 $target$，则说明可以到达。
然而本题中给定的网格规模是 $10^6 \times 10^6$，常规的广度优先搜索会远远超出时间限制，因此必须进行优化。
注意到障碍个数不超过 200 个，说明网格中大部分都不是障碍，只有极少一部分是障碍。分析从 $source$ 无法走到 $target$ 的情况，无非是以下两种。
* $source$ 被障碍完全包围，并且 $target$ 不在包围圈中。
* $target$ 被障碍完全包围，并且 $source$ 不在包围圈中。

在【障碍个数不超过 200 个前提下】，可以猜测包围圈的大小不会很大。也就是说，如果障碍的个数为 n 个，那么我们希望得到包围圈中包含的非障碍位置的数量的上限，假设其为 $limit$，那么：
> 如果我们从 $source$ 开始广度优先搜索，搜索到的非障碍位置数量严格大于 $limit$，那么说明 $source$ 一定不在任何包围圈中。同时，如果从 $target$ 开始广度优先搜索也是如此，那么同样说明 $target$ 一定也不在任何包围圈中。此时可以断定一定能从 $source$ 走到 $target$。

如果目标是正确估计包围圈的上限。当障碍的个数固定位 n 个时（只考虑 $n \geq 2$ 的情况，如果 $n = 0, 1$，无法形成包围圈，任意两个位置都互相可达），要想使得非障碍位置数量最大，这些障碍一定是靠着网格的一个角进行包围的，因为这样可以利用网格的两条边。
不失一般性，假设靠着的是网格的左上角，可以证明，最优的包围方法是选择：
$$(0, n - 1), (1, n - 2), (2, n - 3), \dots , (n - 3, 2), (n - 2, 1), (n - 1, 0)$$
这 n 个位置，此时包围圈组成等腰直角三角形。
<img alt="avatar" height="571" src="https://assets.leetcode-cn.com/solution-static/1036/1.png" width="1000"/>
其中包含的非障碍位置数量为：$\frac{n(n - 1)}{2}$
最优性可以通过构造法证明。可以发现，上面给出的包围方法保证了每一行和每一列不会有超过 1 个障碍。对于其他的包围方法，不失一般性，设某一行有 2 个或以上的障碍，此时有两种情况：
* 如果这两个障碍不八连通，那么其中一个障碍的设置是无意义的，因为它不和包围圈的外界联通，放置该障碍只会减少包围圈中非障碍位置的数量。
* 如果这两个障碍八连通，可以从第一个障碍开始，到离第二个障碍最近的并且行坐标小于等于它为止的所有障碍的行坐标加 1 ，这样可以构造出一个包含非障碍位置数量最多的包围圈。

<img alt="avatar" height="386" src="https://assets.leetcode-cn.com/solution-static/1036/2.png" width="1000"/>

因此，最优的包围方法一定保证每一行和每一列不会有超过 1 个障碍。由于需要包围住左上角，因此列坐标必须取 $0, 1, 2, \dots , n - 1$ 这 n 个位置（如果取 $\geq n$，那么该障碍就无法与包围圈连通，因为它至少还需要借助 n 个障碍来连接到左边界）。同时，每一行只有一个障碍，因此每个障碍右侧的所有位置是不可能被包围的，只有左侧的位置才可能被包围。而左侧位置的总数为：
$$ 0 + 1 + 2 + \dots + (n - 1) = \frac{n(n - 1)}{2} $$
因此最多包含非障碍位置数量为 $\frac{n(n - 1)}{2}$。

### 算法
通过上述证明，可以设计出如下算法：
* 从 $source$ 开始进行广度优先搜索。如果经历了不超过 $\frac{n(n - 1)}{2}$ 个非障碍位置就已经结束搜索，说明 $source$ 在包围圈中。但如果在过程中经过了 $target$，那么说明它们是可达的，否则一定不可达。
* 再从 $target$ 开始进行广度优先搜索。如果经过了不超过 $\frac{n(n - 1)}{2}$ 个非障碍位置就已经结束搜索，说明 $target$ 在包围圈中。否则说明 $source$ 和 $target$ 均不在包围圈中，此时一定可达。

搜索的过程中需要借助哈希表来标记每个位置是否已经搜索过。

In [4]:
import collections
from typing import List
def isEscapePossible(blocked: List[List[int]], source: List[int], target: List[int]) -> bool:
    """
    BLOCKED: 在包围圈中
    VALID:   不在包围圈中
    FOUND:   无论在不在包围圈中，但在 n(n-1)/2 步搜索的过程中经过了 target
    """
    BLOCKED, VALID, FOUND = -1, 0, 1
    BOUNDARY = 10**6
    if len(blocked) < 2:
        return True
    hashBlocked = set(tuple(pos) for pos in blocked)

    def check(start: List[int], finish: List[int]) -> int:
        startX, startY = start
        finishX, finishY = finish
        countdown = len(blocked) * (len(blocked) - 1) // 2
        q = collections.deque([(startX, startY)])
        visited = {(startX, startY)}
        while q and countdown > 0:
            x, y = q.popleft()
            for nextX, nextY in [(x + 1, y), (x - 1, y), (x, y + 1), (x, y - 1)]:
                if 0 <= nextX < BOUNDARY and 0 <= nextY < BOUNDARY and (nextX, nextY) not in hashBlocked and (nextX, nextY) not in visited:
                    if (nextX, nextY) == (finishX, finishY):
                        return FOUND
                    countdown -= 1
                    q.append((nextX, nextY))
                    visited.add((nextX, nextY))
        if countdown > 0:
            return BLOCKED
        return VALID

    result = check(source, target)
    if result == FOUND:
        return True
    elif result == BLOCKED:
        return False
    else:
        result = check(target, source)
        if result == BLOCKED:
            return False
        return True

isEscapePossible([[0,3],[1,0],[1,1],[1,2],[1,3]], [0, 0], [0, 2])

True

Traceback (most recent call last):
  File "_pydevd_bundle\pydevd_cython_win32_38_64.pyx", line 1035, in _pydevd_bundle.pydevd_cython_win32_38_64.PyDBFrame.trace_dispatch
  File "D:\Programs\JetBrains\PyCharm 2021.3\plugins\python\helpers-pro\jupyter_debug\pydev_jupyter_plugin.py", line 144, in cmd_step_over
    if _is_inside_jupyter_cell(frame, pydb):
  File "D:\Programs\JetBrains\PyCharm 2021.3\plugins\python\helpers-pro\jupyter_debug\pydev_jupyter_plugin.py", line 209, in _is_inside_jupyter_cell
    if is_cell_filename(filename):
  File "D:\Programs\JetBrains\PyCharm 2021.3\plugins\python\helpers-pro\jupyter_debug\pydev_jupyter_plugin.py", line 220, in is_cell_filename
    ipython_shell = get_ipython()
NameError: name 'get_ipython' is not defined


NameError: name 'isEscapePossible' is not defined